In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

In [5]:
train = pd.read_csv("train.csv")
test= pd.read_csv("test.csv")
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
X_train = (train.iloc[:,1:].values).astype('float32') # all pixel values
y_train = train.iloc[:,0].values.astype('int32') # only labels i.e targets digits
X_test = test.values.astype('float32')
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
print(X_train.shape)
print(X_test.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [21]:
mean = X_train.mean().astype(np.float32)
std = X_train.std().astype(np.float32)

def standardize (X):
    return  (X - mean) / std

print(X_train.mean())
print(X_train.std())

2.8897537e-08
1.0


In [16]:
y_train= to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

10

In [18]:
def CNN_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
model= CNN_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 26, 26, 32)        104       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 12, 12, 32)        48        
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 10, 10, 64)        18496     
__________

In [26]:
model.fit(X_train, y_train, batch_size=32,epochs=1, verbose=1)

Epoch 1/1
42000/42000 [==============================] - 238s 6ms/step - loss: 0.1157 - acc: 0.9648


In [27]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
model.fit(X_train_, y_train_, batch_size=32, epochs=1, verbose=1, validation_data=(X_test_, y_test_))

Train on 29400 samples, validate on 12600 samples
Epoch 1/1
29400/29400 [==============================] - 192s 7ms/step - loss: 0.0509 - acc: 0.9830 - val_loss: 0.0354 - val_acc: 0.9888


In [28]:
model.fit(X_train_, y_train_, batch_size=16, epochs=20, verbose=1, validation_data=(X_test_, y_test_))

Train on 29400 samples, validate on 12600 samples
Epoch 1/20
29400/29400 [==============================] - 208s 7ms/step - loss: 0.0877 - acc: 0.9736 - val_loss: 0.0328 - val_acc: 0.9886
Epoch 2/20
29400/29400 [==============================] - 209s 7ms/step - loss: 0.0520 - acc: 0.9849 - val_loss: 0.0381 - val_acc: 0.9890
Epoch 3/20
29400/29400 [==============================] - 209s 7ms/step - loss: 0.0500 - acc: 0.9850 - val_loss: 0.0463 - val_acc: 0.9861
Epoch 4/20
29400/29400 [==============================] - 206s 7ms/step - loss: 0.0380 - acc: 0.9890 - val_loss: 0.0307 - val_acc: 0.9898
Epoch 5/20
29400/29400 [==============================] - 206s 7ms/step - loss: 0.0346 - acc: 0.9897 - val_loss: 0.0298 - val_acc: 0.9914
Epoch 6/20
29400/29400 [==============================] - 183s 6ms/step - loss: 0.0317 - acc: 0.9901 - val_loss: 0.0481 - val_acc: 0.9865
Epoch 7/20
29400/29400 [==============================] - 158s 5ms/step - loss: 0.0232 - acc: 0.9925 - val_loss: 0.0359 - 